# Importing libraries

In [1]:
import time
import warnings                           # to ignore runtime warnings
import pandas as pd                       # pandas for data preprocessing
import seaborn as sns                     # seaborn for data visualization
import cufflinks as cf                    # cufflinks for plotly 
import chart_studio.plotly as py
import plotly.express as px               # plotly for advance data visualization
import speech_recognition as sr           # speech_recognition library for taking voice as an input
import matplotlib.pyplot as plt
from plotly.offline import download_plotlyjs , plot , iplot , init_notebook_mode
init_notebook_mode(connected = True)
cf.go_offline()
warnings.filterwarnings('ignore')

# Main Project

In [2]:
''' we start by creating a parent class recurrent phrase
                            and declare the filler words in the constructor of parent class '''


class Recurrent_phrase:     # parent class
    
    def __init__(self):     # parent class constructor
        
        # declaring the filler words
        self.words_to_recognize = [
                        "well",'um','er','uh',"hmm","like","actually","basically","seriously","you see","you know","I mean",
                         "at the end of the day" , "believe me", "I guess" , "I suppose" , "or something" , "ok so" , "right"
                      ]
        
        # creating instance for speech recognition
        self.speech_1 = sr.Recognizer() 


        
''' create a class frequency_check to check the frequency of the filler words if they occur in the sentence '''

class frequency_check(Recurrent_phrase):      # child class frequency_check inherits parent class Recurrent_phrase
    
    def __init__(self):                       # child class constructor
        
        # calling the constructor of parent class
        super().__init__()
        
        
        
        
    ''' we now create a function to check whether the filler words occur in the sentence '''
    
    def character_check(self):                           # defining a function character_check
        self.words = []                                  # creating the list to append the words if they appear
        with sr.Microphone() as source:                  # using Microphone function to take input from the microphone
            start_time = time.time()
            print('say something!!')
            audio = self.speech_1.listen(source)         # using listen function to listen the source
            end_time = time.time()
            self.character = self.speech_1.recognize_google(audio) # using google's API to recognize the words
            for char in self.words_to_recognize:
                if char in self.character:
                    self.words.append(char)              # appending the filler words in the list if they exists in the sentence
                    
            self.total_time = end_time - start_time
            
                    
    ''' we now create a function o check the frequency of those filler words '''
    
    def check_frequency(self):                       # defining a function check_frequency
        self.character_check()
        self.frequency = {}                          # creating a dictionary to store the filler words if they exist
        for i in self.words:
            x = self.character.count(i)
            if i not in self.frequency:
                self.frequency[i] = x
            else:
                self.frequency[i] += 1
        return self.character , self.total_time      # returning the input given by the user
    
    
    
    
''' visualizing the frequency of filler words , here we are using python plotly library for advance data visualization '''

class visualize_frequency(frequency_check):         # child class visualize_frequency inherits the parent chass frequency_check 
    
    def __init__(self):                             # constructor of child class 
        
        # calling the parent class constructor
        super().__init__()
       
    
    # creating a dataset of the filler words
    def data_set(self):
         self.data = pd.DataFrame(pd.Series(self.frequency) , columns = ['Frequency'])
    
    # visualizing the dataset
    def visualize(self):
        self.data_set()
        if len(self.data) != 0:                     # checking whether we have encountered any filler word or not
            
            # creating a bar plot of the frequencies of the filler words using bar plt
            fig = px.bar(data_frame = self.data , x = self.data.index , y = 'Frequency',color = self.data.index)
            fig.show()
        else:
            pass
    
vi = visualize_frequency()
word_spoken , total_time_taken = vi.check_frequency()
print('sentence spoken by the user : ',word_spoken)
print('total time taken by the user : ',total_time_taken)
vi.visualize()

say something!!
sentence spoken by the user :  so basically my name is Sakshi Verma I like to do programming and believe me I am and undergraduate student I guess today was a good day but I suppose this meeting is going great
total time taken by the user :  18.305255889892578
